In [ ]:
def compute_beta_sm(df: pd.DataFrame, stock_col: str, market_col: str) -> float:
    df_clean = df[[stock_col, market_col]].dropna()
    X = sm.add_constant(df_clean[market_col])
    y = df_clean[stock_col]
    model = sm.OLS(y, X).fit()
    return model.params[market_col]  # Beta is the slope

In [217]:
import plotly.express as px
import seaborn as sns
from datetime import datetime, timedelta, date
from typing import List
import plotly.io as pyo
import math
import pandas as pd
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from datetime import datetime, date, timedelta
from xbbg import blp
import blpapi
import os
import win32com.client as win32
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.io import to_html
from plotly.subplots import make_subplots
import statsmodels.api as sm
from plotly.offline import plot
import scipy.optimize as spop
from scipy.stats import t
from scipy.stats import norm
from statsmodels.regression.rolling import RollingOLS
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.colors as mcolors
import sqlite3
from pandas.tseries.offsets import BDay
import scipy.stats as stats

In [240]:
innova_asset_class = r"C:\Users\Admin\OneDrive - Innova Asset Management\Models\James\Performance Excels - Shane\Autoupdated20210207\Attribution\innova_asset_class_returns.xlsx"
file_path = r"C:\Users\Admin\OneDrive - Innova Asset Management\Innova Asset Management Team Site - 2024_Python_Files\Benchmarks_Master_Scoring_2024.xlsx"
innova_asset_class = pd.read_excel(innova_asset_class, engine="openpyxl", index_col=0, sheet_name='Monthly Dates') #'Daily Dates'
innova_asset_class.index = innova_asset_class.index.to_period('M').to_timestamp('M')
innova_asset_class = innova_asset_class[~(innova_asset_class.fillna(0) == 0).all(axis=1)]  # Drop rows where all (non-NaN) values are 0
innova_returns = (innova_asset_class / innova_asset_class.shift()) - 1
global_benchmarks = pd.read_excel(file_path, engine="openpyxl", index_col=0, sheet_name='Global')
aus_benchmarks = pd.read_excel(file_path, engine="openpyxl", index_col=0, sheet_name='Global')
rf_index = global_benchmarks.columns.get_loc("RF")
global_benchmarks = (global_benchmarks.iloc[:, :rf_index+1]).dropna().iloc[:-1]
global_benchmarks['Market'] = global_benchmarks['Market-RF'] + global_benchmarks['RF']
aus_benchmarks = (aus_benchmarks.iloc[:, :rf_index+1]).dropna().iloc[:-1]
aus_benchmarks['Market'] = aus_benchmarks['Market-RF'] + global_benchmarks['RF']

global_benchmarks.index = pd.to_datetime(global_benchmarks.index)
aus_benchmarks.index = pd.to_datetime(aus_benchmarks.index)
global_benchmarks.index = global_benchmarks.index.to_period('M').to_timestamp('M')
aus_benchmarks.index = aus_benchmarks.index.to_period('M').to_timestamp('M')

#GLOBAL MASTER
global_df = pd.concat([innova_returns['Global Eq'], global_benchmarks['Market']], axis = 1, join = 'inner')
aus_df = pd.concat([innova_returns['Aus Eq'], aus_benchmarks['Market']], axis = 1, join = 'inner')

global_df = global_df.dropna()
global_df['Excess'] = global_df['Global Eq'] - global_df['Market']
global_df['up/down'] = np.where(global_df['Market'] > 0, 1, 0)
quartile_labels = ['Low Market Returns', 'Moderate-Low', 'Moderate-High', 'High Market Returns']
global_df['benchmark_quartile'] = pd.qcut(
    global_df['Market'],
    q=4,
    labels=quartile_labels)

aus_df = aus_df.dropna()
aus_df['Excess'] = aus_df['Aus Eq'] - aus_df['Market']
aus_df['up/down'] = np.where(aus_df['Market'] > 0, 1, 0)
quartile_labels = ['Low Market Returns', 'Moderate-Low', 'Moderate-High', 'High Market Returns']
aus_df['benchmark_quartile'] = pd.qcut(
    aus_df['Market'],
    q=4,
    labels=quartile_labels)

Beta Upside Downside:

In [ ]:
start = '2020-05-31'
global_df = global_df.loc[start:]
global_df.groupby('benchmark_quartile')['Excess'].mean()*12
global_df.groupby('up/down')['Excess'].mean()*12
down = global_df[global_df['up/down']==0]
up = global_df[global_df['up/down']==1]
downside_beta = compute_beta_sm(down, 'Global Eq','Market')
upside_beta = compute_beta_sm(up, 'Global Eq','Market')
reg = compute_beta_sm(global_df, 'Global Eq','Market')
d = down.count()['Market']
u = up.count()['Market']
ds = [downside_beta, d]
us = [upside_beta, u]
reg_ = [reg, global_df.count()['Market']]
beta_overall = []
dict = {'Downside':ds, 'Upside':us, 'Total':reg_}
df_du =pd.DataFrame(dict, index = ['Beta','Count'])
ratios = compute_capture_ratios(global_df)
pd.DataFrame(ratios, index = ['Ratio'])

In [294]:
def compute_capture_ratios(df: pd.DataFrame) -> dict:
    df_clean = df[['Global Eq', 'Market']].dropna()
    up = df_clean[df_clean['Market'] > 0]
    up_capture = up['Global Eq'].mean() / up['Market'].mean()
    down = df_clean[df_clean['Market'] < 0]
    down_capture = down['Global Eq'].mean() / down['Market'].mean()
    return {
        'Upside': round(up_capture, 4),
        'Downside': round(down_capture, 4)
    }

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# --- Set Plotly Theme ---
pio.templates["custom"] = go.layout.Template(
    layout=go.Layout(
        font=dict(family="Montserrat", size=14, color="#30415f"),
        title_font=dict(family="Montserrat", size=18, color="#30415f"),
        colorway=["#30415f", "#e9a166", "#62b398"],
        xaxis=dict(showgrid=True, zeroline=False),
        yaxis=dict(showgrid=True, zeroline=False),
        plot_bgcolor="white",
        paper_bgcolor="white"
    )
)
pio.templates.default = "custom"

# --- Helper Functions ---
def geometric_annualised_return(returns, periods_per_year=12):
    cumulative_return = (1 + returns).prod()
    n_periods = len(returns)
    return cumulative_return**(periods_per_year / n_periods) - 1

def geometric_annualised_volatility(returns, periods_per_year=12):
    return returns.std() * np.sqrt(periods_per_year)

def rolling_beta(s, b, w=36):
    return s.rolling(w).cov(b) / b.rolling(w).var()

def rolling_beta_fill(s, b, condition, w=36):
    filled = pd.Series(index=s.index, dtype=float)
    mask = condition.reindex(s.index, fill_value=False)
    for i in range(w - 1, len(s)):
        if mask.iloc[i]:
            s_window = s.iloc[i - w + 1:i + 1]
            b_window = b.iloc[i - w + 1:i + 1]
            if len(s_window) >= w:
                beta_val = np.cov(s_window, b_window)[0, 1] / np.var(b_window)
                filled.iloc[i] = beta_val
    return filled.ffill()

# --- Main Analysis Function ---
def analyze_strategy_geometric(global_df, start_date, end_date):
    df = global_df.loc[start_date:end_date].dropna()
    strategy = df.iloc[:, 0]
    benchmark = df.iloc[:, 1]
    periods_per_year = 12

    # Annualised Stats
    ann_return_strategy = geometric_annualised_return(strategy)
    ann_return_benchmark = geometric_annualised_return(benchmark)
    ann_std_strategy = geometric_annualised_volatility(strategy)
    ann_std_benchmark = geometric_annualised_volatility(benchmark)
    ret_risk_strategy = ann_return_strategy / ann_std_strategy
    ret_risk_benchmark = ann_return_benchmark / ann_std_benchmark

    # Max Drawdown
    def compute_drawdown_series(returns):
        cumulative = (1 + returns).cumprod()
        peak = cumulative.cummax()
        return (cumulative - peak) / peak

    max_dd_strategy = compute_drawdown_series(strategy).min()
    max_dd_benchmark = compute_drawdown_series(benchmark).min()

    # Conditional Returns
    up_mask = benchmark > 0
    down_mask = benchmark <= 0
    avg_excess_ret_up = geometric_annualised_return(strategy[up_mask] - benchmark[up_mask])
    avg_excess_ret_down = geometric_annualised_return(strategy[down_mask] - benchmark[down_mask])

    # Tracking Error & Betas
    tracking_error = (strategy - benchmark).std() * np.sqrt(periods_per_year)
    beta = np.cov(strategy, benchmark)[0, 1] / np.var(benchmark)

    def conditional_beta(s, b, mask):
        s_masked = s[mask]
        b_masked = b[mask]
        if len(s_masked) < 2:
            return np.nan
        return np.cov(s_masked, b_masked)[0, 1] / np.var(b_masked)

    beta_up = conditional_beta(strategy, benchmark, up_mask)
    beta_down = conditional_beta(strategy, benchmark, down_mask)

    std_up_strategy = geometric_annualised_volatility(strategy[up_mask])
    std_down_strategy = geometric_annualised_volatility(strategy[down_mask])
    std_up_benchmark = geometric_annualised_volatility(benchmark[up_mask])
    std_down_benchmark = geometric_annualised_volatility(benchmark[down_mask])

    # Summary Table
    summary_df = pd.DataFrame({
        'Statistic': [
            "Annualised Return", "Annualised Std Dev", "Return / Risk", "Max Drawdown",
            "Avg Excess Ret (Up)", "Avg Excess Ret (Down)", "Tracking Error", "Beta to Benchmark",
            "Upside Beta", "Downside Beta", "Annualised Upside Std Dev", "Annualised Downside Std Dev"
        ],
        'Strategy': [
            ann_return_strategy, ann_std_strategy, ret_risk_strategy, max_dd_strategy,
            avg_excess_ret_up, avg_excess_ret_down, tracking_error, beta,
            beta_up, beta_down, std_up_strategy, std_down_strategy
        ],
        'Benchmark': [
            ann_return_benchmark, ann_std_benchmark, ret_risk_benchmark, max_dd_benchmark,
            0.0, 0.0, np.nan, np.nan, np.nan, np.nan, std_up_benchmark, std_down_benchmark
        ]
    })

    # Rolling Metrics
    rolling_te = (strategy - benchmark).rolling(window=36).std() * np.sqrt(periods_per_year)
    roll_beta = rolling_beta(strategy, benchmark, 36)
    roll_beta_up = rolling_beta_fill(strategy, benchmark, benchmark > 0, 36)
    roll_beta_down = rolling_beta_fill(strategy, benchmark, benchmark <= 0, 36)

    # Recession Analysis
    crisis_periods = {
        "COVID (2020)": ("2020-03-31", "2020-03-31"),
        "Inflation Shock (2022)": ("2022-01-01", "2022-12-31"),
        "2018 Vol Shock": ("2018-10-01", "2018-12-31")
    }

    rows = []
    for label, (s, e) in crisis_periods.items():
        sub = global_df.loc[s:e].dropna()
        strat = sub.iloc[:, 0]
        bench = sub.iloc[:, 1]
        r_strat = geometric_annualised_return(strat)
        r_bench = geometric_annualised_return(bench)
        sd_strat = geometric_annualised_volatility(strat)
        sd_bench = geometric_annualised_volatility(bench)
        rows.append((label, r_strat, sd_strat, r_strat / sd_strat, r_bench, sd_bench, r_bench / sd_bench))

    recession_df = pd.DataFrame(rows, columns=[
        "Period", "Strategy Ann Ret", "Strategy Ann Std Dev", "Strategy Return/Risk",
        "Benchmark Ann Ret", "Benchmark Ann Std Dev", "Benchmark Return/Risk"
    ]).set_index("Period")

    return summary_df, rolling_te, roll_beta, roll_beta_up, roll_beta_down, compute_drawdown_series(strategy), compute_drawdown_series(benchmark), recession_df

def arithmetic_annualised_return(returns, periods_per_year=12):
    return returns.mean() * periods_per_year

def update_avg_excess_to_arithmetic(summary_df, strategy, benchmark, periods_per_year=12):
    # Masks for benchmark direction
    up_mask = benchmark > 0
    down_mask = benchmark <= 0

    # Recalculate Avg Excess Ret (Up/Down) using arithmetic method
    avg_excess_up = arithmetic_annualised_return(strategy[up_mask] - benchmark[up_mask], periods_per_year)
    avg_excess_down = arithmetic_annualised_return(strategy[down_mask] - benchmark[down_mask], periods_per_year)

    # Update the summary DataFrame
    summary_df.loc[summary_df['Statistic'] == "Avg Excess Ret (Up)", 'Strategy'] = avg_excess_up
    summary_df.loc[summary_df['Statistic'] == "Avg Excess Ret (Down)", 'Strategy'] = avg_excess_down

    return summary_df

# --- Plot Functions ---
def plot_drawdown_chart(dd_strategy, dd_benchmark):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dd_strategy.index, y=dd_strategy * 100, name='Strategy', mode='lines'))
    fig.add_trace(go.Scatter(x=dd_benchmark.index, y=dd_benchmark * 100, name='Benchmark', mode='lines'))
    fig.update_layout(
        title='Drawdown History',
        yaxis_title='Drawdown (%)',
        xaxis_title='Date',
        yaxis=dict(ticksuffix='%'),
        hovermode='x unified',
        width=900
    )
    return fig

def plot_rolling_metrics(rolling_te, roll_beta, roll_beta_up, roll_beta_down):
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=rolling_te.index, y=rolling_te * 100, name='Tracking Error', mode='lines'))
    fig1.update_layout(
        title='Rolling 3Y Tracking Error',
        yaxis_title='Tracking Error (%)',
        xaxis_title='Date',
        yaxis=dict(ticksuffix='%'),
        hovermode='x unified',
        width=900
    )

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=roll_beta.index, y=roll_beta, name='Beta', mode='lines'))
    fig2.add_trace(go.Scatter(x=roll_beta_up.index, y=roll_beta_up, name='Upside Beta', mode='lines'))
    fig2.add_trace(go.Scatter(x=roll_beta_down.index, y=roll_beta_down, name='Downside Beta', mode='lines'))
    fig2.update_layout(
        title='Rolling 3Y Beta Metrics',
        yaxis_title='Beta',
        xaxis_title='Date',
        hovermode='x unified',
        width=900
    )
    return fig1, fig2

In [ ]:
start_date = global_df.dropna().first_valid_index()
end_date = global_df.dropna().last_valid_index()
results = analyze_strategy_geometric( global_df, start_date , end_date )
summary_df, rolling_te, roll_beta, roll_beta_up, roll_beta_down, dd_strat, dd_bench, recession_df = results
summary_df = update_avg_excess_to_arithmetic(summary_df, global_df.iloc[:, 0], global_df.iloc[:, 1])
display(summary_df, recession_df)
plot_drawdown_chart(dd_strat, dd_bench).show()
plot_rolling_metrics(rolling_te, roll_beta, roll_beta_up, roll_beta_down)[0].show()
plot_rolling_metrics(rolling_te, roll_beta, roll_beta_up, roll_beta_down)[1].show()